In [1]:
import os
import csv
import pandas as pd
import thesis_tools
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('sm')

import sys
if sys.platform=='darwin':
    top_path='/Users/codydirks/PGCC/'
else:
    top_path='/DataDisk/datafiles/PGCC_HST/'

x1d_dir=top_path+'x1d_files/'

In [2]:
with open('primary_sightlines.csv','r') as csvfile:
    inp=csv.reader(csvfile)
    primary_sightlines=next(inp)

sl_data=thesis_tools.load_data(primary_sightlines)
all_data=thesis_tools.load_results(sl_data)

/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/codydirks/python/thesis_tools/loaders.py:105: Runtim

In [3]:
from astroquery.vizier import Vizier
import astropy.units as u

Vizier.ROW_LIMIT=10000

catalog='J/ApJS/199/8'

ns=['N({:})'.format(ion) for ion in ('H','HI','H2')]
columns=['Name']
for n in ns:
    for flag in ('l_','','E_','e_','r_'):
        columns.append(flag+n)

        
results=pd.DataFrame(columns=columns)
v = Vizier(columns=columns)

new_cols={}
for sl in primary_sightlines:
    rows=v.query_object(sl,catalog=catalog+'/table1',radius=2*u.arcsec)
    if len(rows)>0:
        rows=rows[0].to_pandas()
        for col in list(rows.columns):
            if col != 'Name':
                new_col=col.replace('N_','N(')
                new_col=new_col[0:-1]+')'
                new_cols[col]=new_col
            else:
                new_cols[col]=col
        rows=rows.rename(columns=new_cols)
        rows['Name']=pd.Series([sl]*len(rows),dtype=str)
        results=results.append(rows,ignore_index=True)

In [39]:
#results.to_csv('vizier_results.csv',index=False)

In [4]:
clean_results=pd.DataFrame(columns=results.columns)
complete_rows=results[(np.isfinite(results['N(H)']) & np.isfinite(results['N(HI)']) & np.isfinite(results['N(H2)']))]
#print(results['Name'].unique())
#print(complete_rows['Name'].unique())
#print([i for i in results['Name'].unique() if i not in complete_rows['Name'].unique()])

for idx,row in complete_rows.iterrows():
    # Checks that values are self consistent, i.e. H2+HI ~ Htot
    if abs((2*row['N(H2)']+row['N(HI)'])/row['N(H)'] - 1)<0.1:
        clean_results=clean_results.append(row,ignore_index=True)
        #print((2*row['N(H2)']+row['N(HI)'])/row['N(H)'])
        #print(row[['N(H)','N(HI)','N(H2)']])
        #print()
#clean_results.to_csv('clean_vizier_results.csv',index=False)

In [7]:
clean_results

,Name,l_N(H),N(H),E_N(H),e_N(H),r_N(H),l_N(HI),N(HI),E_N(HI),e_N(HI),r_N(HI),l_N(H2),N(H2),E_N(H2),e_N(H2),r_N(H2)
0,HD23180,b'',1.622000e+21,NaN,NaN,b'58',b'',7.943000e+20,2.800000e+20,NaN,"b'23, 32'",b'',3.981000e+20,NaN,NaN,"b'23, 32, 34, 288'"
1,HD23180,b'',1.610000e+21,NaN,NaN,"b'23, 61'",b'',7.080000e+20,1.200000e+20,NaN,"b'22, 32'",b'',4.100000e+20,NaN,NaN,b'60'
2,HD23478,b'',1.622000e+21,1.600000e+20,NaN,b'38',b'',1.023000e+21,1.500000e+20,NaN,b'38',b'',3.020000e+20,5.300000e+19,NaN,b'38'
3,HD24398,b'',1.585000e+21,NaN,NaN,"b'40, 58'",b'',6.400000e+20,6.000000e+19,NaN,b'43',b'',4.700000e+20,2.000000e+20,NaN,"b'18, 238'"
4,HD24398,b'',1.580000e+21,4.600000e+20,NaN,b'18',b'',6.456000e+20,6.200000e+19,NaN,"b'19, 23, 32, 41'",b'',4.677000e+20,NaN,NaN,"b'23, 32, 34, 64, 70, 81, 113, 288'"
5,HD24398,b'',1.585000e+21,2.300000e+20,NaN,"b'19, 41'",b'',6.310000e+20,NaN,NaN,"b'34, 493'",b'',4.700000e+20,NaN,NaN,"b'23,60, 61, 288'"
6,HD24398,b'',1.580000e+21,1.900000e+20,NaN,b'69',b'',6.500000e+20,NaN,NaN,b'60',b'',4.677000e+20,1.200000e+20,NaN,"b'19, 41'"
7,HD24398,b'',1.580000e+21,NaN,NaN,"b'23, 61, 288'",b'',6.456000e+20,1.300000e+20,NaN,b'74',b'',4.700000e+20,1.200000e+20,NaN,b'43'
8,HD24534,b'',2.188000e+21,1.600000e+20,NaN,"b'27, 29, 35'",b'',3.981000e+20,NaN,NaN,b'-999',b'',8.318000e+20,8.000000e+19,NaN,"b'11, 24, 27, 29, 35, 37, 95, 591'"
9,HD24534,b'',2.200000e+21,2.200000e+20,NaN,b'18',b'',5.370000e+20,8.000000e+19,NaN,"b'11, 27, 29, 35, 74'",b'',8.320000e+20,7.300000e+19,NaN,"b'11, 18'"
